In [ ]:
import cv2
import pandas as pd
from PIL import Image
import math

## 環境建置

In [ ]:
# clone
!git clone https://github.com/PaddlePaddle/PaddleOCR

# install gpu元件
!python -m pip install paddlepaddle-gpu==2.1.3.post101 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
!pip install "paddleocr>=2.0.1"

# 安裝所需套件
!cd /content/drive/MyDrive/PaddleOCR;pip3 install -r requirements.txt

In [ ]:
# 測試是否建置成功
import paddle
paddle.utils.run_check()

## PPOCR 訓練資料準備

In [ ]:
# 裁切圖片 (另有針對額外預測正反面信心水準較低之圖片進行翻轉) 此處僅裁切的示意程式,包含人工無法以程式表示

# 匯入訓練標註資料
df = pd.read_csv(r'/content/drive/MyDrive/t_brain/Adam/public_training_data_adam.csv')

# 依資料給定長寬
df = df.assign(weight = lambda d : (d['xmax']-d['xmin'])*1.0 )
df = df.assign(height = lambda d : (d['ymax']-d['ymin'])*1.0 )

for _,pic in df.iterrows():
# 讀取圖檔
  print(f"{pic['filename']}")
  img = cv2.imread(f"/content/drive/MyDrive/t_brain/public_training_data/{pic['filename']}.jpg")

# 裁切區域的 x 與 y 座標（左上角） : 改取min可能更好
  x = math.ceil(pic['xmin'])
  if x < 0:
    x = 0
  y = math.ceil(pic['ymin'])
# 裁切區域的長度與寬度
  w = math.ceil(pic['weight'])
  h = math.ceil(pic['height'])

# 裁切圖片
  crop_img = img[y:y+h, x:x+w]
# 寫出圖片
  cv2.imwrite(f"/content/drive/MyDrive/t_brain/full_cut_v2/cut_{pic['filename']}.jpg",crop_img)

## Paddle Training
    需準備yml檔 / yml檔案中指定之訓練資料路徑txt檔
    需將yml檔放至指定資料夾，或修改路徑 (目前上傳版的設定檔皆放置於_HiAdam_configs)
    
    下載pretrained model 至 PaddleOCR/Adam/pretrain
    https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_server_v2.0_rec_pre.tar

In [ ]:
# version a training
!cd /content/drive/MyDrive/PaddleOCR; python3 tools/train.py -c configs/rec/ch_ppocr_v2.0/rec_chinese_common_train_v2.0_adam_a.yml

In [ ]:
# version b training
!cd /content/drive/MyDrive/PaddleOCR; python3 tools/train.py -c configs/rec/ch_ppocr_v2.0/rec_chinese_common_train_v2.0_adam_b.yml

In [ ]:
# version c training
!cd /content/drive/MyDrive/PaddleOCR; python3 tools/train.py -c configs/rec/ch_ppocr_v2.0/rec_chinese_common_train_v2.0_adam_c.yml

In [ ]:
# predict modle 轉 infer model -> save至相對應的資料夾 (資料夾的位置需做相對應的調整)
!cd /content/drive/MyDrive/PaddleOCR;python3 tools/export_model.py -c configs/rec/ch_ppocr_v2.0/rec_chinese_common_train_v2.0_adam_a.yml -o Global.pretrained_model=/content/drive/MyDrive/PaddleOCR/Adam/output/rec/common_a/best_accuracy Global.save_inference_dir=./Adam/output/rec/common_a/inference
!cd /content/drive/MyDrive/PaddleOCR;python3 tools/export_model.py -c configs/rec/ch_ppocr_v2.0/rec_chinese_common_train_v2.0_adam_b.yml -o Global.pretrained_model=/content/drive/MyDrive/PaddleOCR/Adam/output/rec/common_b/best_accuracy Global.save_inference_dir=./Adam/output/rec/common_b/inference
!cd /content/drive/MyDrive/PaddleOCR;python3 tools/export_model.py -c configs/rec/ch_ppocr_v2.0/rec_chinese_common_train_v2.0_adam_c.yml -o Global.pretrained_model=/content/drive/MyDrive/PaddleOCR/Adam/output/rec/common_c/best_accuracy Global.save_inference_dir=./Adam/output/rec/common_c/inference


## Paddle cls 方向模型訓練
    需準備 yml設定檔 / 訓練圖片位置txt檔
    均需將設定檔擺入指定的資料夾

In [ ]:
# training
!cd /content/drive/MyDrive/PaddleOCR;python3 tools/train.py -c configs/cls/cls_mv3_adam.yml

In [ ]:
# 轉inference cls
!cd /content/drive/MyDrive/PaddleOCR;python3 tools/export_model.py -c configs/cls/cls_mv3_adam.yml -o Global.pretrained_model=./Adam/output/cls/v0/best_accuracy Global.save_inference_dir=./Adam/output/cls/v0/inference